In [2]:
import argparse
import os
import sys
import pickle
from pytorch_transformers import BertTokenizer
from collections import Counter, defaultdict
from tqdm.auto import tqdm
from itertools import product
from gensim.models import KeyedVectors
from gensim.similarities import SoftCosineSimilarity
import numpy as np
from scipy.spatial.distance import cosine
import random

parser = argparse.ArgumentParser()
parser.add_argument("--data_home", type=str, default="/ssd2/arthur/TREC2019/data")
parser.add_argument("--docs_file", type=str, default="docs/tokenized-msmarco-docs.tsv")
parser.add_argument("--queries_file", type=str, default="queries/test_queries.tsv")
parser.add_argument("--axioms", type=str, default="TFC1,TFC2,MTDC,LNC1,LNC2,LB1,LB2,STMC1,STMC2,STMC3,TP")
parser.add_argument("--top100", type=str, default="runs/indri_test_10_10.run")
parser.add_argument("--cpus", type=int, default=1)
parser.add_argument("--total_docs", type=int, default=3213835)
parser.add_argument("--delta", type=int, default=10)
parser.add_argument("--idf_file", type=str, default="docs/IDFS/IDFS-FULL")
parser.add_argument("--embeddings_path", type=str, default="GloVe/w2v.txt")
parser.add_argument("--stmc_sim", type=float, default=0.2)
parser.add_argument("--delta_lb", type=float, default=0.2)
parser.add_argument("--min_df_lb", type=int, default=100000)

argv = ["--cpus", "1",
        "--axioms", "TFC1"]
args = parser.parse_args(argv)
sys.path.insert(0, "/ssd2/arthur/TREC2019/scripts/")

In [3]:
def getcontent(doc_id, docs_file, offset_dict):
    offset = offset_dict[doc_id]
    with open(docs_file) as f:
        f.seek(offset)
        doc = f.readline()
    return doc

docs_path = os.path.join(args.data_home, args.docs_file)
offset_dict = pickle.load(open(docs_path+".offset", 'rb'))
top_100_path = os.path.join(args.data_home, args.top100)
assert os.path.isfile(top_100_path)
assert os.path.isfile(docs_path)
#set of valid documents
tuples = defaultdict(lambda: set())
all_docs = {}
docs_lens = {}
scores = {}
for i in tqdm(open(top_100_path), total=155800):
    topic_id, _ , doc_id, _, score, _ = i.split()
    tuples[topic_id].add(doc_id)
    scores[f"{topic_id}-{doc_id}"] = score
    if doc_id in all_docs:
        continue  
    all_docs[doc_id] = getcontent(doc_id, docs_path, offset_dict).split()
    docs_lens[doc_id] = len(all_docs[doc_id])
args.scores = scores

In [15]:
# create a new trec_parameter file.https://33d36b12.ngrok.io/notebooks/LNC_ALL%20dataset.ipynb
# 
args.delta_lb = 0.01
import re
import random
def LB1(topic_id, tokenized_query, all_docs, tuples, docs_lens, args):
    query = [x.replace("##", "") for x in tokenized_query]
    query_terms = set(query)
    instances = []
    _docs = list(tuples[topic_id])
    formatter = "<query>\n<number>{}</number>\n<text>#combine({})</text>\n{}\n</query>\n"
    for i, di_id in enumerate(_docs):
        di_terms = set(all_docs[di_id])
        di_terms_not_q = di_terms.difference(query_terms) 
        score_di = float(args.scores[f"{topic_id}-{di_id}"])
        if not os.path.isfile(os.path.join(args.data_home, "queries/LB1.indriparam")):
            with open(os.path.join(args.data_home, "queries/LB1.indriparam"), 'w') as outf:
                outf.write("<parameters>\n<threads> 48 </threads>\n")
        with open(os.path.join(args.data_home, "queries/LB1.indriparam"), 'a') as outf:
            for dj_id in _docs[i:]:
                score_dj = float(args.scores[f"{topic_id}-{dj_id}"])
                score_diff = abs(float(args.scores[f"{topic_id}-{di_id}"])-float(args.scores[f"{topic_id}-{dj_id}"]))
                try:
                    score_diff /= max(float(args.scores[f"{topic_id}-{di_id}"]), float(args.scores[f"{topic_id}-{dj_id}"]))
                except:
                # normalization broke this example. both are actually the exact same score.
                    if max(float(args.scores[f"{topic_id}-{di_id}"]), float(args.scores[f"{topic_id}-{dj_id}"])) == 0:
                        score_diff = 0
                if score_diff >= args.delta_lb:
                    continue
                dj_terms = set(all_docs[dj_id])
                dj_terms_not_q = dj_terms.difference(query_terms)
                # pick a term from Dj, that is not in Di, and add to query.
                dj_unique_terms = dj_terms_not_q.difference(di_terms_not_q)
                if len(dj_unique_terms) !=0:
                    random_dj_term = random.sample(dj_unique_terms, 1)
                    #avoid non-alpha (indri is not happy with it)
                    while re.match('^[\w-]+$', random_dj_term[0]) is None:
                        random_dj_term = random.sample(dj_unique_terms, 1)
                    q_dj = " ".join(query + random_dj_term)
                    q_dj = re.sub(r'([^\s\w]|_)+', '', q_dj)

                    q_dj_id = topic_id+"-"+dj_id+"-"+di_id # in this case, the query was modified to help dj
                    instances.append((q_dj_id, dj_id, di_id)) 
                    working_set = "<workingSetDocno>{}</workingSetDocno>\n<workingSetDocno>{}</workingSetDocno>".format(di_id, dj_id)
                    query_format_j = formatter.format(q_dj_id, q_dj, working_set)
                    outf.write(query_format_j)

                di_unique_terms = di_terms_not_q.difference(dj_terms_not_q)
                if len(di_unique_terms) != 0:
                    random_di_term = random.sample(di_unique_terms, 1)
                    # Avoid non-alphanumeric
                    while re.match('^[\w-]+$', random_di_term[0]) is None:
                        random_di_term = random.sample(di_unique_terms, 1)
                    q_di = " ".join(query + random_di_term)
                    q_di = re.sub(r'([^\s\w]|_)+', '', q_di)
                    if len(q_di) == 0:
                        print("FIND")
                    q_di_id = topic_id+"-"+di_id+"-"+dj_id # in this case, the query was modified to help di
                    instances.append((q_di_id, di_id, dj_id)) 
                    query_format_i = formatter.format(q_di_id, q_di, working_set)
                    outf.write(query_format_i)
    return instances

queries_file = os.path.join(args.data_home, args.queries_file)
assert os.path.isfile(queries_file)
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

instances = []
try:
    os.remove("/ssd2/arthur/TREC2019/data/queries/LB1.indriparam")
except:
    pass
for counter, line in tqdm(enumerate(open(queries_file)), total=1558):
    q_id, query_text = line.strip().split("\t")
    tokenized_query = tokenizer.tokenize(query_text)
    instances += LB1(q_id, tokenized_query, all_docs, tuples, docs_lens, args)
    if counter > 1:
        break
with open(os.path.join(args.data_home, "queries/LB1.indriparam"), 'a') as outf:
    outf.write("</parameters>")
pickle.dump(instances, open("/ssd2/arthur/TREC2019/data/diagnostics/LB1-instances", 'wb'))

indri_path = "/ssd2/arthur/indri/bin/IndriRunQuery"
param_file = "/ssd2/arthur/TREC2019/data/queries/LB1.indriparam"
indri_cmd = "{} -index=/ssd2/arthur/TREC2019/data/index/indri-tokenized-bert-index/ {} -trecFormat=true > /ssd2/arthur/TREC2019/data/runs/LB1.run".format(indri_path, param_file)
subprocess.check_output(indri_cmd.split())

CalledProcessError: Command '['/ssd2/arthur/indri/bin/IndriRunQuery', '-index=/ssd2/arthur/TREC2019/data/index/indri-tokenized-bert-index/', '/ssd2/arthur/TREC2019/data/queries/LB1.indriparam', '-trecFormat=true', '>', '/ssd2/arthur/TREC2019/data/runs/LB1.run']' returned non-zero exit status 255.

In [12]:
indri_cmd

'/ssd2/arthur/indri/bin/IndriRunQuery -index=/ssd2/arthur/TREC2019/data/index/indri-tokenized-bert-index/ /ssd2/arthur/TREC2019/data/queries/LB1.indriparam -trecFormat=true > /ssd2/arthur/TREC2019/data/runs/LB1.run'

In [5]:
pickle.dump(instances, open("/ssd2/arthur/TREC2019/data/diagnostics/LB1-instances", 'wb'))

In [4]:
indri_cmd

'/ssd2/arthur/indri/bin/IndriRunQuery -index=/ssd2/arthur/TREC2019/data/index/indri-tokenized-bert-index/ /ssd2/arthur/TREC2019/data/queries/LB1.indriparam -trecFormat=true > /ssd2/arthur/TREC2019/data/runs/LB1.run'

In [6]:
# Run indri with the new indriparam file
import subprocess
indri_path = "/ssd2/arthur/indri/bin/IndriRunQuery"
param_file = "/ssd2/arthur/TREC2019/data/queries/LB1.indriparam"
indri_cmd = "{} -index=/ssd2/arthur/TREC2019/data/index/indri-tokenized-bert-index/ {} -trecFormat=true > /ssd2/arthur/TREC2019/data/runs/LB1.run".format(indri_path, param_file)
subprocess.run(indri_cmd.split())

CompletedProcess(args=['/ssd2/arthur/indri/bin/IndriRunQuery', '-index=/ssd2/arthur/TREC2019/data/index/indri-tokenized-bert-index/', '/ssd2/arthur/TREC2019/data/queries/LB1.indriparam', '-trecFormat=true', '>', '/ssd2/arthur/TREC2019/data/runs/LB1.run'], returncode=255)

In [5]:
instances = pickle.load(open("/ssd2/arthur/TREC2019/data/diagnostics/LB1-instances", 'rb'))

In [8]:
from pytorch_transformers import DistilBertForSequenceClassification
model = DistilBertForSequenceClassification.from_pretrained("/ssd2/arthur/TREC2019/data/models/distilbert-cut/")

In [10]:
def text_to_tokens(query, document):
    tokens_a = query
    tokens_b = document
    _truncate_seq_pair(tokens_a, tokens_b)
    tokens = ["[CLS]"] + tokens_a + ["[SEP]"] + tokens_b + ["[SEP]"]
    return tokens
def _truncate_seq_pair(tokens_a, tokens_b, max_length=509):
    """Truncates a sequence pair in place to the maximum length."""
    while True:
        total_length = len(tokens_a) + len(tokens_b)
        if total_length <= max_length:
            break
        if len(tokens_a) > len(tokens_b):
            tokens_a.pop()
        else:
            tokens_b.pop()

In [11]:
buffer = ""
with open("/ssd2/arthur/TREC2019/data/triples-tokenized/LB1", 'w') as outf:
    for counter, line in tqdm(enumerate(open("/ssd2/arthur/TREC2019/data/queries/LB1.indriparam")), total=92545202):
        if "</query>" in line:
            topic_id, dj, di = buffer.split("<number>")[1].split("</")[0].split("-")
            query = tokenizer.tokenize(buffer.split("#combine(")[1].split(")<")[0])
            pair_id = buffer.split("<number>")[1].split("</")[0]
            di_doc = all_docs[di]
            di_triple = text_to_tokens(query, di_doc)
            outf.write("{}\t{}\n".format(pair_id, di_triple))
            
            dj = buffer.split("Docno>")[3].split("</")[0]
            dj_doc = all_docs[di]
            dj_triple = text_to_tokens(query, dj_doc)
            outf.write("{}\t{}\n".format(pair_id, dj_triple))
            buffer = ""
        else:
            buffer += line

In [11]:
from pytorch_transformers import DistilBertForSequenceClassification
model = DistilBertForSequenceClassification.from_pretrained("/ssd2/arthur/TREC2019/data/models/distilbert-cut/")
batch_size = 1024
from msmarco_dataset import MsMarcoDataset
from torch.utils.data import DataLoader
dataset = MsMarcoDataset("/ssd2/arthur/TREC2019/data/triples-tokenized/LB1", args.data_home, distil=True, invert_label=True, labeled=False)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)§b

In [ ]:
import torch
softmax = torch.nn.Softmax(dim=1)

alphas = np.arange(0.0, 1.0)
runs_format = "{} Q0 {} {} {} DISTILBERT_QL\n" #topic_id, doc_id, ranking, score
ql_run_file = os.path.join("/ssd2/arthur/TREC2019/data/runs/LB1.run")
preds = softmax(torch.as_tensor(_preds))[:,0].cpu().numpy()
for alpha in alphas:
    print(alpha)
    beta = 1-alpha
    out_run_file = os.path.join("/ssd2/arthur/TREC2019/data/runs/LB1_distilBert-{}.run".format(alpha))
    topic_results = [] 
    last_topic = -1
    with open(ql_run_file, 'r') as inf, open(out_run_file, 'w') as outf:
        for counter, (example, score) in enumerate(zip(inf, preds)):
            topic_id, _, doc_id, _, _, _ = example.split()
            guid = "{}-{}".format(topic_id, doc_id)
            if topic_id != last_topic and len(topic_results) > 0:
                topic_results.sort(key=lambda x:x['score'], reverse=True)
                for rank, topic in enumerate(topic_results):
                    outf.write(runs_format.format(topic['topic_id'], topic['doc_id'], rank, topic['score']))
                topic_results = []
            topic_results.append({'topic_id':topic_id, 'doc_id':doc_id, 'score':alpha*score + beta*ql_scores[guid]})
            last_topic = topic_id
        topic_results.sort(key=lambda x:x['score'], reverse=True)
        for rank, topic in enumerate(topic_results):
            outf.write(runs_format.format(topic['topic_id'], topic['doc_id'], rank, topic['score']))

In [11]:
import re

instances = []
def LB2(topic_id, tokenized_query, all_docs, tuples, docs_lens, args):
    working_set_f = "<workingSetDocno>{}</workingSetDocno>\n<workingSetDocno>{}</workingSetDocno>"
    formatter = "<query>\n<number>{}</number>\n<text>#combine({})</text>\n{}\n</query>\n"
    DFs = pickle.load(open(os.path.join(args.data_home, args.idf_file),'rb'))
    query = [x.replace("##", "") for x in tokenized_query]
    query_terms = set(query)
    query_terms_with_high_idfs = set([w for w in query_terms if DFs[w]<args.min_df_lb])
    instances = []
    docs_created = dict()
    _docs = list(tuples[topic_id])
    #consider only terms with DF < 100k
    for i, di_id in enumerate(_docs):
        di_terms = set(all_docs[di_id])
        di_query_terms = query_terms.intersection(di_terms)
        di_query_terms_idfs = {w:DFs[w] for w in di_query_terms if DFs[w]<args.min_df_lb}
        # This document already has every possible query term. 
        if set(di_query_terms_idfs.keys()) == query_terms_with_high_idfs:
            continue
        missing_query_terms = query_terms_with_high_idfs.difference(di_query_terms_idfs.keys())
        score_di = float(args.scores[f"{topic_id}-{di_id}"])
        for dj_id in _docs[i:]:
            if di_id == dj_id:
                continue
            # Documents score must be similar
            score_diff = abs(float(args.scores[f"{topic_id}-{di_id}"])-float(args.scores[f"{topic_id}-{dj_id}"]))
            try:
                score_diff /= max(float(args.scores[f"{topic_id}-{di_id}"]), float(args.scores[f"{topic_id}-{dj_id}"]))
            except:
                # normalization broke this example. both are actually the exact same score.
                if max(float(args.scores[f"{topic_id}-{di_id}"]), float(args.scores[f"{topic_id}-{dj_id}"])) == 0:
                    score_diff = 0
            if score_diff > args.delta_lb:
                continue
            dj_terms = set(all_docs[dj_id])
            dj_query_terms = query_terms.intersection(dj_terms)
            dj_query_terms_idfs = {w:DFs[w] for w in dj_query_terms if DFs[w]<args.min_df_lb}
            
            # This document already has every possible query term. 
            if set(dj_query_terms_idfs.keys()) == query_terms_with_high_idfs:
                continue

            # Docs must contain the same query terms overall
            if di_query_terms_idfs != dj_query_terms_idfs:
                continue

            #Pick a query term PAIR with similar IDF
            valid_term_pairs = []
            for q1, q2 in product(dj_query_terms_idfs.keys(), missing_query_terms):
                if IDFs[q1] == 0:
                    continue
                difference = abs(IDFs[q1]-IDFs[q2])/IDFs[q1]
                if difference < args.delta_lb:
                    valid_term_pairs.append((q1, q2))
            # Create new documents. Add q2 to D2, repeat q1 in D1. Score for D2 must become larger than D1.
            # Add to start of document to avoid BERT issues.
            for q1, q2 in valid_term_pairs:
                di_prime = [q1] + all_docs[di_id]
                di_prime_id = di_id + "-" + q1 
                dj_prime = [q2] + all_docs[dj_id]
                dj_prime_id = dj_id + "-" + q2
                docs_created[di_prime_id] = di_prime
                docs_created[dj_prime_id] = dj_prime
                instances.append((topic_id, dj_prime_id, di_prime_id))

                # Do the opposite now.
                dj_prime = [q1] + all_docs[dj_id]
                dj_prime_id = dj_id + "-" + q1
                di_prime = [q2] + all_docs[di_id]
                di_prime_id = di_id + "-" + q2
                docs_created[di_prime_id] = di_prime
                docs_created[dj_prime_id] = dj_prime
                instances.append((topic_id, di_prime_id, dj_prime_id))
    # Write documents to file
    if len(docs_created) == 0:
        return []
    trectext_format = "<DOC>\n<DOCNO>{}</DOCNO>\n<TEXT>{}\n</TEXT>\n</DOC>\n"
    with open("/ssd2/arthur/TREC2019/data/docs/LB2/{}.trectext".format(topic_id), 'w') as outf:
        for d_id in docs_created:
            outf.write((trectext_format.format(d_id, docs_created[d_id])))
    # Write queries to file
    if not os.path.isfile("/ssd2/arthur/TREC2019/data/queries/LB2.indriparam"):
        with open("/ssd2/arthur/TREC2019/data/queries/LB2.indriparam", 'w') as outf:
            outf.write("<parameters>\n<threads> 48 </threads>\n")
    with open("/ssd2/arthur/TREC2019/data/queries/LB2.indriparam", 'a') as outf: 
        for _, di, dj in instances:
            working_set = working_set_f.format(di, dj)
            _query = re.sub(r'([^\s\w]|_)+', '', " ".join(query))
            outf.write(formatter.format(topic_id, _query, working_set))
    return instances

os.remove("/ssd2/arthur/TREC2019/data/queries/LB2.indriparam")
for line in tqdm(open(queries_file), total=1558):
    q_id, query_text = line.strip().split("\t")
    tokenized_query = tokenizer.tokenize(query_text)
    instances += LB2(q_id, tokenized_query, all_docs, tuples, docs_lens, args)
with open("/ssd2/arthur/TREC2019/data/queries/LB2.indriparam", 'a') as outf: 
    outf.write("</parameters>")

with open("/ssd2/arthur/TREC2019/data/diagnostics/LB2-instances" , 'wb') as outf:
    pickle.dump(instances, outf)

In [12]:
instances = pickle.load(open("/ssd2/arthur/TREC2019/data/diagnostics/LB2-instances", 'rb'))

In [13]:
LB2_scores = {}
run_file = "/ssd2/arthur/TREC2019/data/runs/LB2.run"
for line in open(run_file):
#     print(line)
    topic_id, _, doc_id, _, score, _ = line.split()
    pair_id = "{}-{}".format(topic_id, doc_id)
    LB2_scores[pair_id] = float(score)

In [19]:
dataset_path = os.path.join(args.data_home, "diagnostics/LB2-instances")
dataset = pickle.load(open(dataset_path, 'rb'))
agreements = 0
for topic_id, di_id, dj_id in dataset:
    guid1 = f"{topic_id}-{di_id}"
    guid2 = f"{topic_id}-{dj_id}"
    if LB2_scores[guid1] > LB2_scores[guid2]:
        agreements+=1
len(dataset), agreements, agreements / len(dataset)

(89642, 84167, 0.9389237187925303)

In [46]:
buffer.split("Docno>")[3].split("</")[0].split("-")[0]

'D3342241'

In [61]:


text_to_tokens(query, di_doc)

In [68]:
# LB2 on BERT
# create triples file based on trec params file
buffer = ""
with open("/ssd2/arthur/TREC2019/data/triples-tokenized/LB2", 'w') as outf:
    for counter, line in tqdm(enumerate(open("/ssd2/arthur/TREC2019/data/queries/LB2.indriparam"))):
        if "</query>" in line:
            topic_id = buffer.split("<number>")[1].split("</")[0]
            query = tokenized_queries[topic_id]
            
            di, extra_token = buffer.split("Docno>")[1].split("</")[0].split("-")
            pair_id = "{}-{}-{}".format(topic_id, di, extra_token)
            if extra_token not in query:
                extra_token = "##"+extra_token
            if extra_token not in query:
                print("WTF")
            new_di = [extra_token] + all_docs[di]
            di_triple = text_to_tokens(query, new_di)
            outf.write("{}\t{}\n".format(pair_id, di_triple))
            
            dj, extra_token = buffer.split("Docno>")[3].split("</")[0].split("-")
            pair_id = "{}-{}-{}".format(topic_id, dj, extra_token)
            if extra_token not in query:
                extra_token = "##"+extra_token
            if extra_token not in query:
                print("WTF")
            new_dj = [extra_token] + all_docs[dj]
            dj_triple = text_to_tokens(query, new_dj)
            outf.write("{}\t{}\n".format(pair_id, dj_triple))
            buffer = ""
        else:
            buffer += line

In [69]:
from pytorch_transformers import DistilBertForSequenceClassification
model = DistilBertForSequenceClassification.from_pretrained("/ssd2/arthur/TREC2019/data/models/distilbert-cut/")

In [ ]:
args.data_home

In [80]:
sys.path.insert(0, "/ssd2/arthur/TREC2019/scripts/")
batch_size = 1024
from msmarco_dataset import MsMarcoDataset
from torch.utils.data import DataLoader
dataset = MsMarcoDataset("/ssd2/arthur/TREC2019/data/triples-tokenized/LB2", args.data_home, distil=True, invert_label=True, labeled=False)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

In [78]:
import torch
seed = 42
gpus_to_use = [0, 1, 3, 6, 7]
torch.cuda.manual_seed_all(seed)
n_gpu = torch.cuda.device_count()
device = torch.device("cuda" if (torch.cuda.is_available() and n_gpu > 0) else "cpu")
model = torch.nn.DataParallel(model, device_ids=gpus_to_use)
model = model.to(device)

In [81]:
import torch
softmax = torch.nn.Softmax(dim=1)

preds = {}
eval_loss = 0.0
nb_eval_steps = 0
preds[dataset] = None
out_label_ids = 0
_preds = None
for index, batch in tqdm(enumerate(dataloader), desc="{} Dataset".format(dataset), total = len(dataloader)):
    model.eval()
    with torch.no_grad():
        inputs = {'input_ids': batch[0].to(device),
                  'attention_mask': batch[1].to(device)}
        outputs = model(**inputs)
        logits = outputs[0]
        if _preds is None:
            _preds = logits.detach().cpu().numpy()
        else:
            batch_predictions = logits.detach().cpu().numpy()
            _preds = np.append(_preds, batch_predictions, axis=0)
torch.save(_preds, os.path.join(args.data_home, 'predictions', 'LB2_.distilBERT.tensor'))

NameError: name 'data_dir' is not defined

In [87]:
len(_preds)

179284

In [108]:
# load QL scores and normalize
from collections import defaultdict
ql_scores = defaultdict(lambda:[])
ordered_topics = []
scores_per_topic = defaultdict(lambda:[])
QL_run_file = "/ssd2/arthur/TREC2019/data/runs/LB2.run"
last_topic = None
normalized_scores = []
with open(QL_run_file, 'r') as inf:
    for counter, line in tqdm(enumerate(inf), desc="reading run file", total=len(dataset)):
        [topic_id, _, doc_id, _, score, _] = line.split()
        if topic_id not in ordered_topics:
            ordered_topics.append(topic_id)
        scores_per_topic[topic_id].append((doc_id, score))
assert sum([len(scores_per_topic[x]) for x in scores_per_topic]) == len(_preds)
#normalize
for _id in tqdm(scores_per_topic, desc="normalizing"):
    _scores = np.asarray([float(x[1]) for x in scores_per_topic[_id]])
    normalized_scores = (_scores - np.min(_scores))/np.ptp(_scores)
    for (did, _), score in zip(scores_per_topic[_id], normalized_scores):
        guid = "{}-{}".format(_id, did)
        ql_scores[guid] = score

In [122]:
alphas = [0.85, 1.0]
runs_format = "{} Q0 {} {} {} DISTILBERT_QL\n" #topic_id, doc_id, ranking, score
ql_run_file = os.path.join("/ssd2/arthur/TREC2019/data/runs/LB2.run")
preds = softmax(torch.as_tensor(_preds))[:,0].cpu().numpy()
for alpha in alphas:
    beta = 1-alpha
    out_run_file = os.path.join("/ssd2/arthur/TREC2019/data/runs/LB2_distilBert-{}.run".format(alpha))
    topic_results = [] 
    last_topic = -1
    with open(ql_run_file, 'r') as inf, open(out_run_file, 'w') as outf:
        for counter, (example, score) in enumerate(zip(inf, preds)):
            topic_id, _, doc_id, _, _, _ = example.split()
            guid = "{}-{}".format(topic_id, doc_id)
            if topic_id != last_topic and len(topic_results) > 0:
                topic_results.sort(key=lambda x:x['score'], reverse=True)
                for rank, topic in enumerate(topic_results):
                    outf.write(runs_format.format(topic['topic_id'], topic['doc_id'], rank, topic['score']))
                topic_results = []
            topic_results.append({'topic_id':topic_id, 'doc_id':doc_id, 'score':alpha*score + beta*ql_scores[guid]})
            last_topic = topic_id
        topic_results.sort(key=lambda x:x['score'], reverse=True)
        for rank, topic in enumerate(topic_results):
            outf.write(runs_format.format(topic['topic_id'], topic['doc_id'], rank, topic['score']))

In [124]:
alphas = [0.0, 0.85, 1.0]
for alpha in alphas:
    LB2_scores = {}
    run_file = "/ssd2/arthur/TREC2019/data/runs/LB2_distilBert-{}.run".format(alpha)
    for line in open(run_file):
        topic_id, _, doc_id, _, score, _ = line.split()
        pair_id = "{}-{}".format(topic_id, doc_id)
        LB2_scores[pair_id] = float(score)
    dataset_path = os.path.join(args.data_home, "diagnostics/LB2-instances")
    dataset = pickle.load(open(dataset_path, 'rb'))
    agreements = 0
    for topic_id, di_id, dj_id in dataset:
        guid1 = f"{topic_id}-{di_id}"
        guid2 = f"{topic_id}-{dj_id}"
        if LB2_scores[guid1] > LB2_scores[guid2]:
            agreements+=1
    print(alpha, len(dataset), agreements, agreements / len(dataset))

0.0 89642 84167 0.9389237187925303
0.85 89642 78655 0.8774346846344347
1.0 89642 43679 0.48726043595635976


In [42]:
tokenized_queries = {}
for line in tqdm(open(queries_file), total=1558):
    q_id, query_text = line.strip().split("\t")
    tokenized_query = tokenizer.tokenize(query_text)
    tokenized_queries[q_id] = tokenized_query


In [84]:
#LB1
instances = pickle.load(open("/ssd2/arthur/TREC2019/data/diagnostics/LB1-instances", 'rb'))
buffer = ""
with open("/ssd2/arthur/TREC2019/data/triples-tokenized/LB1", 'w') as outf:
    for counter, line in tqdm(enumerate(open("/ssd2/arthur/TREC2019/data/queries/LB1.indriparam2")), total=10730450):
        if "</query>" in line:
            topic_id, dj, di = buffer.split("<number>")[1].split("</")[0].split("-")
            query = tokenizer.tokenize(buffer.split("#combine(")[1].split(")<")[0])
            pair_id = buffer.split("<number>")[1].split("</")[0]
            di_doc = all_docs[di]
            di_triple = text_to_tokens(query, di_doc)
            outf.write("{}\t{}\n".format(pair_id, di_triple))
            
            dj = buffer.split("Docno>")[3].split("</")[0]
            dj_doc = all_docs[di]
            dj_triple = text_to_tokens(query, dj_doc)
            outf.write("{}\t{}\n".format(pair_id, dj_triple))
            buffer = ""
        else:
            buffer += line

In [92]:
from pytorch_transformers import DistilBertForSequenceClassification
model = DistilBertForSequenceClassification.from_pretrained("/ssd2/arthur/TREC2019/data/models/distilbert-cut/")
batch_size = 1024
from msmarco_dataset import MsMarcoDataset
from torch.utils.data import DataLoader
# dataset = MsMarcoDataset("/ssd2/arthur/TREC2019/data/triples-tokenized/LB1", args.data_home, distil=True, invert_label=True, labeled=False)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

In [93]:
import torch
seed = 42
gpus_to_use = [0, 1, 3, 5, 6]
torch.cuda.manual_seed_all(seed)
n_gpu = torch.cuda.device_count()
device = torch.device("cuda" if (torch.cuda.is_available() and n_gpu > 0) else "cpu")
model = torch.nn.DataParallel(model, device_ids=gpus_to_use)
model = model.to(device)

In [94]:
import torch
softmax = torch.nn.Softmax(dim=1)

preds = {}
eval_loss = 0.0
nb_eval_steps = 0
preds[dataset] = None
out_label_ids = 0
_preds = None
for index, batch in tqdm(enumerate(dataloader), desc="{} Dataset".format(dataset), total = len(dataloader)):
    model.eval()
    with torch.no_grad():
        inputs = {'input_ids': batch[0].to(device),
                  'attention_mask': batch[1].to(device)}
        outputs = model(**inputs)
        logits = outputs[0]
        if _preds is None:
            _preds = logits.detach().cpu().numpy()
        else:
            batch_predictions = logits.detach().cpu().numpy()
            _preds = np.append(_preds, batch_predictions, axis=0)
torch.save(_preds, os.path.join(args.data_home, 'predictions', 'LB1_.distilBERT.tensor'))

In [13]:
import torch
_preds = torch.load(os.path.join(args.data_home, 'predictions', 'LB1_.distilBERT.tensor'))

In [11]:
batch_size = 1024
from msmarco_dataset import MsMarcoDataset
from torch.utils.data import DataLoader
dataset = MsMarcoDataset("/ssd2/arthur/TREC2019/data/triples-tokenized/LB1", args.data_home, distil=True, invert_label=True, labeled=False)
# dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

In [24]:
from collections import defaultdict
ql_scores = defaultdict(lambda:[])
ordered_topics = []
scores_per_topic = defaultdict(lambda:[])
QL_run_file = "/ssd2/arthur/TREC2019/data/runs/LB1.run2"
last_topic = None
normalized_scores = []
# pre-load keys. Will make the growth fase of the dictionary faster! (Nice tip!)
# all_keys = list()
# for line in tqdm(open(QL_run_file, 'r'), total=len(dataset)):
#     all_keys.append(line.split()[0])

    
# all_keys = set(all_keys)  

# scores_per_topic = dict.fromkeys(all_keys)
with open(QL_run_file, 'r') as inf:
    for counter, line in tqdm(enumerate(inf), desc="reading run file", total=len(dataset)):
        [topic_id, _, doc_id, _, score, _] = line.split()
        if topic_id not in scores_per_topic:
            ordered_topics.append(topic_id)
        try:
            scores_per_topic[topic_id].append((doc_id, score))
        except:
            scores_per_topic[topic_id] = [] 
            scores_per_topic[topic_id].append((doc_id, score))
assert sum([len(scores_per_topic[x]) for x in scores_per_topic]) == len(_preds)
# normalize
for _id in tqdm(scores_per_topic, desc="normalizing"):
    _scores = np.asarray([float(x[1]) for x in scores_per_topic[_id]])
    normalized_scores = (_scores - np.min(_scores))/np.ptp(_scores)
    for (did, _), score in zip(scores_per_topic[_id], normalized_scores):
        guid = "{}-{}".format(_id, did)
        ql_scores[guid] = score

/home/arthur/miniconda3/envs/bert/lib/python3.7/site-packages/ipykernel_launcher.py:31: RuntimeWarning: invalid value encountered in true_divide


In [35]:
import numpy as np
np.arange(0, 1, 0.05)

array([0.  , 0.05, 0.1 , 0.15, 0.2 , 0.25, 0.3 , 0.35, 0.4 , 0.45, 0.5 ,
       0.55, 0.6 , 0.65, 0.7 , 0.75, 0.8 , 0.85, 0.9 , 0.95])

In [39]:
import torch
softmax = torch.nn.Softmax(dim=1)

alphas = np.arange(0.95, 1.05, 0.05)
runs_format = "{} Q0 {} {} {} DISTILBERT_QL\n" #topic_id, doc_id, ranking, score
ql_run_file = os.path.join("/ssd2/arthur/TREC2019/data/runs/LB1.run2")
preds = softmax(torch.as_tensor(_preds))[:,0].cpu().numpy()
for alpha in alphas:
    print(alpha)
    beta = 1-alpha
    out_run_file = os.path.join("/ssd2/arthur/TREC2019/data/runs/LB1_distilBert-{}.run".format(alpha))
    topic_results = [] 
    last_topic = -1
    with open(ql_run_file, 'r') as inf, open(out_run_file, 'w') as outf:
        for counter, (example, score) in enumerate(zip(inf, preds)):
            topic_id, _, doc_id, _, _, _ = example.split()
            guid = "{}-{}".format(topic_id, doc_id)
            if topic_id != last_topic and len(topic_results) > 0:
                topic_results.sort(key=lambda x:x['score'], reverse=True)
                for rank, topic in enumerate(topic_results):
                    outf.write(runs_format.format(topic['topic_id'], topic['doc_id'], rank, topic['score']))
                topic_results = []
            topic_results.append({'topic_id':topic_id, 'doc_id':doc_id, 'score':alpha*score + beta*ql_scores[guid]})
            last_topic = topic_id
        topic_results.sort(key=lambda x:x['score'], reverse=True)
        for rank, topic in enumerate(topic_results):
            outf.write(runs_format.format(topic['topic_id'], topic['doc_id'], rank, topic['score']))

0.95
1.0
1.05


In [54]:
alphas = [1.0]
for alpha in alphas:
    LB1_scores = {}
    run_file = "/ssd2/arthur/TREC2019/data/runs/LB1_distilBert-{}.run".format(alpha)
    for line in open(run_file):
        topic_id, _, doc_id, _, score, _ = line.split()
        pair_id = "{}-{}".format(topic_id, doc_id)
        LB1_scores[pair_id] = float(score)
    dataset_path = os.path.join(args.data_home, "diagnostics/LB1-instances")
    dataset = pickle.load(open(dataset_path, 'rb'))
    agreements = 0
    for topic_id, di_id, dj_id in dataset:
        guid1 = f"{topic_id}-{di_id}"
        guid2 = f"{topic_id}-{dj_id}"
        print(LB1_scores[guid1], LB1_scores[guid2])
        if LB1_scores[guid1] >= LB1_scores[guid2]:
            agreements+=1
    print(alpha, len(dataset), agreements, agreements / len(dataset))

0.0005734225851483643 0.0005734225851483643
0.1915506273508072 0.1915506273508072
0.0006296511855907738 0.0006296511855907738
0.006296343170106411 0.006296343170106411
0.0007116979104466736 0.0007116979104466736
2.4944336473708972e-05 2.4944336473708972e-05
0.0003654638712760061 0.0003654638712760061
0.0022488024551421404 0.0022488024551421404
0.0003014164976775646 0.0003014164976775646
0.00042772843153215945 0.00042772843153215945
0.0004893834120593965 0.0004893834120593965
0.4114294648170471 0.4114294648170471
0.006836877670139074 0.006836877670139074
0.00012900662841275334 0.00012900662841275334
0.003710501827299595 0.003710501827299595
6.533959094667807e-05 6.533959094667807e-05
0.005533524323254824 0.005533524323254824
0.04466388002038002 0.04466388002038002
0.014779866673052311 0.014779866673052311
0.00021240560454316437 0.00021240560454316437
0.004494111053645611 0.004494111053645611
9.989992395276204e-05 9.989992395276204e-05
0.003169981762766838 0.003169981762766838
0.00151615

0.14539475739002228 0.14539475739002228
0.030954310670495033 0.030954310670495033
0.1756039261817932 0.1756039261817932
0.1876656860113144 0.1876656860113144
0.1922445148229599 0.1922445148229599
0.03548819199204445 0.03548819199204445
0.1381203681230545 0.1381203681230545
0.09593021124601364 0.09593021124601364
0.19532907009124756 0.19532907009124756
0.20348988473415375 0.20348988473415375
0.11936572194099426 0.11936572194099426
0.08674150705337524 0.08674150705337524
0.12443513423204422 0.12443513423204422
0.07764073461294174 0.07764073461294174
0.17207340896129608 0.17207340896129608
0.03647048398852348 0.03647048398852348
0.13929219543933868 0.13929219543933868
0.1273449808359146 0.1273449808359146
0.17423732578754425 0.17423732578754425
0.006424020044505596 0.006424020044505596
0.1416301429271698 0.1416301429271698
0.1072847917675972 0.1072847917675972
0.21081259846687317 0.21081259846687317
0.23243048787117004 0.23243048787117004
0.17366336286067963 0.17366336286067963
0.04138884

0.00029521287069655955 0.00029521287069655955
0.0017964928410947323 0.0017964928410947323
0.00019997914205305278 0.00019997914205305278
0.0013807839713990688 0.0013807839713990688
0.00011087559687439352 0.00011087559687439352
0.00035705266054719687 0.00035705266054719687
0.0002388746797805652 0.0002388746797805652
0.004821377340704203 0.004821377340704203
0.00021862174617126584 0.00021862174617126584
0.0021990956738591194 0.0021990956738591194
0.00016579624207224697 0.00016579624207224697
0.0018329123267903924 0.0018329123267903924
0.0011490547331050038 0.0011490547331050038
0.0021907715126872063 0.0021907715126872063
0.0010061076609417796 0.0010061076609417796
0.0007535050390288234 0.0007535050390288234
0.00014779462071601301 0.00014779462071601301
0.003763108979910612 0.003763108979910612
8.772790897637606e-05 8.772790897637606e-05
0.0012801679549738765 0.0012801679549738765
3.798388934228569e-05 3.798388934228569e-05
0.00035961094545200467 0.00035961094545200467
0.000415040907682850

0.0010052722645923495 0.0010052722645923495
0.0006513610132969916 0.0006513610132969916
0.0016779620200395584 0.0016779620200395584
0.0006513610132969916 0.0006513610132969916
0.0003921438183169812 0.0003921438183169812
0.003694778075441718 0.003694778075441718
0.0002948421170003712 0.0002948421170003712
0.0014280001632869244 0.0014280001632869244
0.0002033612399827689 0.0002033612399827689
0.0004314585239626467 0.0004314585239626467
0.0006938741425983608 0.0006938741425983608
0.0015732066240161657 0.0015732066240161657
0.003941099159419537 0.003941099159419537
0.0019900857005268335 0.0019900857005268335
6.981768819969147e-05 6.981768819969147e-05
0.00239932956174016 0.00239932956174016
0.0014587575569748878 0.0014587575569748878
0.0017865365371108055 0.0017865365371108055
0.0017506685107946396 0.0017506685107946396
0.003064302494749427 0.003064302494749427
0.0005876405048184097 0.0005876405048184097
0.000971702509559691 0.000971702509559691
5.986703035887331e-05 5.986703035887331e-05


8.590646757511422e-05 8.590646757511422e-05
0.0008851105812937021 0.0008851105812937021
0.0011094613000750542 0.0011094613000750542
0.0003270354063715786 0.0003270354063715786
0.015566809102892876 0.015566809102892876
0.0001426610251655802 0.0001426610251655802
6.420929275918752e-05 6.420929275918752e-05
0.00072383665246889 0.00072383665246889
5.772812073701061e-05 5.772812073701061e-05
0.0001259857672266662 0.0001259857672266662
6.497467256849632e-05 6.497467256849632e-05
0.00012166143278591335 0.00012166143278591335
0.0006681226659566164 0.0006681226659566164
0.000680830969940871 0.000680830969940871
2.3067472284310497e-05 2.3067472284310497e-05
0.00016206511645577848 0.00016206511645577848
4.172677654423751e-05 4.172677654423751e-05
0.00023693867842666805 0.00023693867842666805
4.4323991460260004e-05 4.4323991460260004e-05
0.00032193431979976594 0.00032193431979976594
0.0012122668558731675 0.0012122668558731675
5.2370749472174793e-05 5.2370749472174793e-05
7.811729301465675e-05 7.81

...

3.4210042940685526e-05 3.4210042940685526e-05
0.0001627768942853436 0.0001627768942853436
3.430079232202843e-05 3.430079232202843e-05
9.084379416890442e-05 9.084379416890442e-05
0.004148965235799551 0.004148965235799551
4.3139298213645816e-05 4.3139298213645816e-05
0.00014440270024351776 0.00014440270024351776
6.591271085198969e-05 6.591271085198969e-05
8.66886621224694e-05 8.66886621224694e-05
6.318953091977164e-05 6.318953091977164e-05
0.00032172779901884496 0.00032172779901884496
7.985098636709154e-05 7.985098636709154e-05
0.00041702386806719005 0.00041702386806719005
2.850334203685634e-05 2.850334203685634e-05
1.405546299793059e-05 1.405546299793059e-05
2.1614108845824376e-05 2.1614108845824376e-05
5.7008888688869774e-05 5.7008888688869774e-05
4.4253589294385165e-05 4.4253589294385165e-05
0.005848505534231663 0.005848505534231663
4.326264388510026e-05 4.326264388510026e-05
0.03368731215596199 0.03368731215596199
5.856663119629957e-05 5.856663119629957e-05
0.0004108798748347908 0.00

0.5807787775993347 0.5807787775993347
0.037866901606321335 0.037866901606321335
0.20490127801895142 0.20490127801895142
0.07481811940670013 0.07481811940670013
0.38685494661331177 0.38685494661331177
0.04780882969498634 0.04780882969498634
0.18067213892936707 0.18067213892936707
0.1448117196559906 0.1448117196559906
0.023038048297166824 0.023038048297166824
0.09690249711275101 0.09690249711275101
0.13072919845581055 0.13072919845581055
0.1157441958785057 0.1157441958785057
0.2707311511039734 0.2707311511039734
0.0774817019701004 0.0774817019701004
0.5887908339500427 0.5887908339500427
0.2077673226594925 0.2077673226594925
0.4605218172073364 0.4605218172073364
0.10199650377035141 0.10199650377035141
0.2704858183860779 0.2704858183860779
0.023477701470255852 0.023477701470255852
0.336325466632843 0.336325466632843
0.3784654140472412 0.3784654140472412
0.034891072660684586 0.034891072660684586
0.1650690883398056 0.1650690883398056
0.15770681202411652 0.15770681202411652
0.1044241786003112

0.015221922658383846 0.015221922658383846
0.0006432416848838329 0.0006432416848838329
0.00011916951189050451 0.00011916951189050451
0.00038773147389292717 0.00038773147389292717
0.05303335189819336 0.05303335189819336
0.0008674286655150354 0.0008674286655150354
0.00034081056946888566 0.00034081056946888566
0.0005976001848466694 0.0005976001848466694
0.01168323215097189 0.01168323215097189
0.0009250662405975163 0.0009250662405975163
0.08052187412977219 0.08052187412977219
0.009921119548380375 0.009921119548380375
0.24890154600143433 0.24890154600143433
0.026306983083486557 0.026306983083486557
0.0006816019886173308 0.0006816019886173308
0.06110655888915062 0.06110655888915062
0.01572369411587715 0.01572369411587715
0.022677293047308922 0.022677293047308922
0.0014501865953207016 0.0014501865953207016
0.007438991218805313 0.007438991218805313
0.002960620913654566 0.002960620913654566
0.01935473456978798 0.01935473456978798
0.03221660479903221 0.03221660479903221
0.012876014225184917 0.012

0.00012158280878793448 0.00012158280878793448
5.907493323320523e-05 5.907493323320523e-05
0.00018707520212046802 0.00018707520212046802
5.21255424246192e-06 5.21255424246192e-06
0.0008059273823164403 0.0008059273823164403
0.0005657117580994964 0.0005657117580994964
0.0012837982503697276 0.0012837982503697276
1.2026783224428073e-05 1.2026783224428073e-05
0.00032095948699861765 0.00032095948699861765
5.458214218378998e-06 5.458214218378998e-06
4.1036591937881894e-06 4.1036591937881894e-06
0.0001400690816808492 0.0001400690816808492
3.929393642465584e-06 3.929393642465584e-06
1.2781269106199034e-05 1.2781269106199034e-05
4.2633550947357435e-06 4.2633550947357435e-06
5.65549044040381e-06 5.65549044040381e-06
5.226367193245096e-06 5.226367193245096e-06
2.7139478333992884e-05 2.7139478333992884e-05
9.638347364671063e-06 9.638347364671063e-06
0.0006422761944122612 0.0006422761944122612
0.0045026205480098724 0.0045026205480098724
9.393918298883364e-05 9.393918298883364e-05
0.001246601808816194

KeyboardInterrupt: 

In [52]:
guid1 == guid2

False